In [1]:
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

for gpu in tf.config.experimental.list_physical_devices('GPU'):
    tf.config.experimental.set_memory_growth(gpu, True)


In [2]:
class Inception(layers.Layer):
    def __init__(self, c1, c2, c3, c4):
        super().__init__()
        # route 1: 1x1 conv
        self.p1_1 = keras.layers.Conv2D(c1, kernel_size=1, padding='same', activation='relu')
        
        # route 2: 1x1 conv & 3x3 conv
        self.p2_1 = keras.layers.Conv2D(c2[0], kernel_size=1, padding='same', activation='relu')
        self.p2_2 = keras.layers.Conv2D(c2[1], kernel_size=3, padding='same', activation='relu')
        
        # route 3: 1x1 conv & 5x5 conv
        self.p3_1 = keras.layers.Conv2D(c3[0], kernel_size=1, padding='same', activation='relu')
        self.p3_2 = keras.layers.Conv2D(c3[1], kernel_size=5, padding='same', activation='relu')
        
        # route 4: 3x3 pool & 1x1 conv
        self.p4_1 = keras.layers.MaxPool2D(pool_size=3, strides=1, padding='same')
        self.p4_2 = keras.layers.Conv2D(c4, kernel_size=1, padding='same', activation='relu')
        return 
    
    def call(self, x):
        p1 = self.p1_1(x)
        p2 = self.p2_2(self.p2_1(x))
        p3 = self.p3_2(self.p3_1(x))
        p4 = self.p4_2(self.p4_1(x))
        return tf.concat([p1, p2, p3, p4], axis=-1)


In [3]:
blk_1 = keras.Sequential([
    layers.Conv2D(64, kernel_size=7, strides=2, padding='same', activation='relu'), 
    layers.MaxPool2D(pool_size=3, strides=2, padding='same'), 
])

blk_2 = keras.Sequential([
    layers.Conv2D(64, kernel_size=1, padding='same', activation='relu'), 
    layers.Conv2D(64*3, kernel_size=3, padding='same', activation='relu'), 
    layers.MaxPool2D(pool_size=3, strides=2, padding='same'), 
])

blk_3 = keras.Sequential([
    Inception(64, (96, 128), (16, 32), 32), 
    Inception(128, (128, 192), (32, 96), 64), 
    layers.MaxPool2D(pool_size=3, strides=2, padding='same'), 
])

blk_4 = keras.Sequential([
    Inception(192, (96, 208), (16, 48), 64), 
    Inception(160, (112, 224), (24, 64), 64), 
    Inception(128, (128, 256), (24, 64), 64), 
    Inception(112, (144, 288), (32, 64), 64), 
    Inception(256, (160, 320), (32, 128), 128), 
    layers.MaxPool2D(pool_size=3, strides=2, padding='same'), 
])

blk_5 = keras.Sequential([
    Inception(256, (160, 320), (32, 128), 128), 
    Inception(384, (192, 384), (48, 128), 128), 
    layers.GlobalAveragePooling2D(), 
])

net = keras.Sequential([
    blk_1, blk_2, blk_3, blk_4, blk_5, 
    layers.Dense(10),
])

In [4]:
input_shape = (1, 96, 96, 1)
X = tf.random.uniform(input_shape)
for blk in net.layers:
    X = blk(X)
    print(blk.name, '\t', X.shape)

net.build(input_shape=input_shape)
print(net.summary())

sequential 	 (1, 24, 24, 64)
sequential_1 	 (1, 12, 12, 192)
sequential_2 	 (1, 6, 6, 480)
sequential_3 	 (1, 3, 3, 832)
sequential_4 	 (1, 1024)
dense 	 (1, 10)
Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      multiple                  3200      
_________________________________________________________________
sequential_1 (Sequential)    multiple                  114944    
_________________________________________________________________
sequential_2 (Sequential)    multiple                  552432    
_________________________________________________________________
sequential_3 (Sequential)    multiple                  2809168   
_________________________________________________________________
sequential_4 (Sequential)    multiple                  2487536   
_________________________________________________________________
dense (Dense)           

In [7]:
class DataLoader():
    def __init__(self):
        fashion_mnist = tf.keras.datasets.fashion_mnist
        (self.X_train, self.y_train), (self.X_test, self.y_test) = fashion_mnist.load_data()
        self.X_train = np.expand_dims(self.X_train.astype(np.float32)/255.0,axis=-1)
        self.X_test = np.expand_dims(self.X_test.astype(np.float32)/255.0,axis=-1)
        self.y_train = self.y_train.astype(np.int32)
        self.y_test = self.y_test.astype(np.int32)
        self.num_train, self.num_test = self.X_train.shape[0], self.X_test.shape[0]

    def get_batch_train(self, batch_size):
        index = np.random.randint(0, self.num_train, batch_size)
        #need to resize images to (224,224)
        resized_images = tf.image.resize_with_pad(self.X_train[index],224,224,)
        return resized_images.numpy(), self.y_train[index]

    def get_batch_test(self, batch_size):
        index = np.random.randint(0, self.num_test, batch_size)
        #need to resize images to (224,224)
        resized_images = tf.image.resize_with_pad(self.X_test[index],224,224,)
        return resized_images.numpy(), self.y_test[index]

batch_size = 64
dataLoader = DataLoader()
x_batch, y_batch = dataLoader.get_batch_train(batch_size)
print("x_batch shape:",x_batch.shape,"y_batch shape:", y_batch.shape)


x_batch shape: (64, 224, 224, 1) y_batch shape: (64,)


In [8]:
net.compile(
    optimizer=keras.optimizers.Adam(1e-7), 
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy']
)

epochs = 1
num_iter = dataLoader.num_train // batch_size
for ep in range(epochs):
    for n in range(num_iter):
        X_batch, y_batch = dataLoader.get_batch_train(batch_size)
        net.fit(X_batch, y_batch)
        if n % 20 == 0:
            net.save_weights('googlenet_weights.h5')
            

Train on 64 samples
64/64 [==============================] - 6s 94ms/sample - loss: 6.9363 - accuracy: 0.0469
Train on 64 samples
64/64 [==============================] - 0s 8ms/sample - loss: 5.0958 - accuracy: 0.1406
Train on 64 samples
64/64 [==============================] - 0s 8ms/sample - loss: 5.7693 - accuracy: 0.2031
Train on 64 samples
64/64 [==============================] - 0s 8ms/sample - loss: 6.0151 - accuracy: 0.0625
Train on 64 samples
64/64 [==============================] - 0s 8ms/sample - loss: 6.0223 - accuracy: 0.0938
Train on 64 samples
64/64 [==============================] - 0s 8ms/sample - loss: 6.5859 - accuracy: 0.0781
Train on 64 samples
64/64 [==============================] - 0s 8ms/sample - loss: 6.3650 - accuracy: 0.0781
Train on 64 samples
64/64 [==============================] - 0s 8ms/sample - loss: 6.6523 - accuracy: 0.0625
Train on 64 samples
64/64 [==============================] - 0s 8ms/sample - loss: 6.9913 - accuracy: 0.1094
Train on 64 sample

64/64 [==============================] - 0s 8ms/sample - loss: 3.8742 - accuracy: 0.0625
Train on 64 samples
64/64 [==============================] - 0s 8ms/sample - loss: 4.6146 - accuracy: 0.0781
Train on 64 samples
64/64 [==============================] - 0s 8ms/sample - loss: 4.5138 - accuracy: 0.0625
Train on 64 samples
64/64 [==============================] - 0s 8ms/sample - loss: 5.1795 - accuracy: 0.0781
Train on 64 samples
64/64 [==============================] - 0s 8ms/sample - loss: 4.5634 - accuracy: 0.1406
Train on 64 samples
64/64 [==============================] - 0s 8ms/sample - loss: 5.4876 - accuracy: 0.1094
Train on 64 samples
64/64 [==============================] - 0s 8ms/sample - loss: 4.8659 - accuracy: 0.1250
Train on 64 samples
64/64 [==============================] - 0s 8ms/sample - loss: 5.2006 - accuracy: 0.0938
Train on 64 samples
64/64 [==============================] - 0s 8ms/sample - loss: 4.4613 - accuracy: 0.1562
Train on 64 samples
64/64 [============

64/64 [==============================] - 0s 8ms/sample - loss: 5.3033 - accuracy: 0.1094
Train on 64 samples
64/64 [==============================] - 0s 8ms/sample - loss: 4.0041 - accuracy: 0.1406
Train on 64 samples
64/64 [==============================] - 0s 8ms/sample - loss: 4.4896 - accuracy: 0.1406
Train on 64 samples
64/64 [==============================] - 0s 8ms/sample - loss: 5.5081 - accuracy: 0.1250
Train on 64 samples
64/64 [==============================] - 0s 8ms/sample - loss: 4.4580 - accuracy: 0.0938
Train on 64 samples
64/64 [==============================] - 0s 8ms/sample - loss: 3.8703 - accuracy: 0.1250
Train on 64 samples
64/64 [==============================] - 0s 8ms/sample - loss: 4.6752 - accuracy: 0.1094
Train on 64 samples
64/64 [==============================] - 0s 8ms/sample - loss: 5.0609 - accuracy: 0.1094
Train on 64 samples
64/64 [==============================] - 0s 8ms/sample - loss: 4.5691 - accuracy: 0.0625
Train on 64 samples
64/64 [============

64/64 [==============================] - 0s 8ms/sample - loss: 5.2375 - accuracy: 0.1250
Train on 64 samples
64/64 [==============================] - 0s 8ms/sample - loss: 5.0691 - accuracy: 0.0938
Train on 64 samples
64/64 [==============================] - 0s 8ms/sample - loss: 4.0898 - accuracy: 0.1875
Train on 64 samples
64/64 [==============================] - 1s 8ms/sample - loss: 4.0350 - accuracy: 0.1094
Train on 64 samples
64/64 [==============================] - 0s 8ms/sample - loss: 4.0178 - accuracy: 0.1875
Train on 64 samples
64/64 [==============================] - 0s 8ms/sample - loss: 5.1528 - accuracy: 0.0469
Train on 64 samples
64/64 [==============================] - 0s 8ms/sample - loss: 4.0042 - accuracy: 0.1094
Train on 64 samples
64/64 [==============================] - 0s 8ms/sample - loss: 4.8519 - accuracy: 0.0625
Train on 64 samples
64/64 [==============================] - 0s 8ms/sample - loss: 4.6509 - accuracy: 0.0625
Train on 64 samples
64/64 [============

64/64 [==============================] - 0s 8ms/sample - loss: 4.7004 - accuracy: 0.0781
Train on 64 samples
64/64 [==============================] - 0s 8ms/sample - loss: 5.2792 - accuracy: 0.0781
Train on 64 samples
64/64 [==============================] - 0s 8ms/sample - loss: 4.7221 - accuracy: 0.1250
Train on 64 samples
64/64 [==============================] - 0s 8ms/sample - loss: 4.6194 - accuracy: 0.0625
Train on 64 samples
64/64 [==============================] - 0s 8ms/sample - loss: 4.2545 - accuracy: 0.0625
Train on 64 samples
64/64 [==============================] - 0s 8ms/sample - loss: 5.1453 - accuracy: 0.0469
Train on 64 samples
64/64 [==============================] - 0s 8ms/sample - loss: 5.2136 - accuracy: 0.0781
Train on 64 samples
64/64 [==============================] - 0s 8ms/sample - loss: 5.0553 - accuracy: 0.1406
Train on 64 samples
64/64 [==============================] - 0s 8ms/sample - loss: 5.3095 - accuracy: 0.1094
Train on 64 samples
64/64 [============

...

64/64 [==============================] - 0s 8ms/sample - loss: 3.9957 - accuracy: 0.1094
Train on 64 samples
64/64 [==============================] - 0s 8ms/sample - loss: 4.2620 - accuracy: 0.1719
Train on 64 samples
64/64 [==============================] - 0s 8ms/sample - loss: 4.3142 - accuracy: 0.0625
Train on 64 samples
64/64 [==============================] - 0s 8ms/sample - loss: 5.1442 - accuracy: 0.0938
Train on 64 samples
64/64 [==============================] - 1s 8ms/sample - loss: 4.8691 - accuracy: 0.0938
Train on 64 samples
64/64 [==============================] - 1s 8ms/sample - loss: 5.2714 - accuracy: 0.0781
Train on 64 samples
64/64 [==============================] - 0s 8ms/sample - loss: 4.4435 - accuracy: 0.1094
Train on 64 samples
64/64 [==============================] - 1s 8ms/sample - loss: 4.4808 - accuracy: 0.0469
Train on 64 samples
64/64 [==============================] - 0s 8ms/sample - loss: 5.5919 - accuracy: 0.0938
Train on 64 samples
64/64 [============

64/64 [==============================] - 0s 8ms/sample - loss: 4.7782 - accuracy: 0.1406
Train on 64 samples
64/64 [==============================] - 0s 8ms/sample - loss: 4.2170 - accuracy: 0.1094
Train on 64 samples
64/64 [==============================] - 0s 8ms/sample - loss: 3.9406 - accuracy: 0.2031
Train on 64 samples
64/64 [==============================] - 0s 8ms/sample - loss: 4.9555 - accuracy: 0.0938
Train on 64 samples
64/64 [==============================] - 1s 8ms/sample - loss: 3.9366 - accuracy: 0.1250
Train on 64 samples
64/64 [==============================] - 0s 8ms/sample - loss: 3.9645 - accuracy: 0.1094
Train on 64 samples
64/64 [==============================] - 0s 8ms/sample - loss: 4.1808 - accuracy: 0.1094
Train on 64 samples
64/64 [==============================] - 0s 8ms/sample - loss: 4.5683 - accuracy: 0.1250
Train on 64 samples
64/64 [==============================] - 0s 8ms/sample - loss: 3.9077 - accuracy: 0.1562
Train on 64 samples
64/64 [============

64/64 [==============================] - 1s 8ms/sample - loss: 4.4781 - accuracy: 0.2188
Train on 64 samples
64/64 [==============================] - 0s 8ms/sample - loss: 3.0741 - accuracy: 0.2344
Train on 64 samples
64/64 [==============================] - 0s 8ms/sample - loss: 4.0104 - accuracy: 0.2031
Train on 64 samples
64/64 [==============================] - 1s 8ms/sample - loss: 3.4286 - accuracy: 0.2344
Train on 64 samples
64/64 [==============================] - 0s 8ms/sample - loss: 3.5983 - accuracy: 0.1875
Train on 64 samples
64/64 [==============================] - 0s 8ms/sample - loss: 4.6049 - accuracy: 0.1562
Train on 64 samples
64/64 [==============================] - 1s 8ms/sample - loss: 4.6106 - accuracy: 0.2188
Train on 64 samples
64/64 [==============================] - 0s 8ms/sample - loss: 4.4700 - accuracy: 0.1250
Train on 64 samples
64/64 [==============================] - 1s 8ms/sample - loss: 4.1093 - accuracy: 0.2031
Train on 64 samples
64/64 [============

64/64 [==============================] - 1s 8ms/sample - loss: 3.7675 - accuracy: 0.1875
Train on 64 samples
64/64 [==============================] - 1s 8ms/sample - loss: 3.4041 - accuracy: 0.2031
Train on 64 samples
64/64 [==============================] - 0s 8ms/sample - loss: 3.9986 - accuracy: 0.2344
Train on 64 samples
64/64 [==============================] - 0s 8ms/sample - loss: 4.5735 - accuracy: 0.1250
Train on 64 samples
64/64 [==============================] - 1s 8ms/sample - loss: 3.5641 - accuracy: 0.2969
Train on 64 samples
64/64 [==============================] - 0s 8ms/sample - loss: 3.5563 - accuracy: 0.2812
Train on 64 samples
64/64 [==============================] - 1s 8ms/sample - loss: 3.4039 - accuracy: 0.2031
Train on 64 samples
64/64 [==============================] - 0s 8ms/sample - loss: 3.9347 - accuracy: 0.2969
Train on 64 samples
64/64 [==============================] - 0s 8ms/sample - loss: 4.3594 - accuracy: 0.2188
Train on 64 samples
64/64 [============

64/64 [==============================] - 0s 8ms/sample - loss: 2.3050 - accuracy: 0.2500
Train on 64 samples
64/64 [==============================] - 0s 8ms/sample - loss: 2.3625 - accuracy: 0.2656
Train on 64 samples
64/64 [==============================] - 1s 8ms/sample - loss: 2.2667 - accuracy: 0.1406
Train on 64 samples
64/64 [==============================] - 0s 8ms/sample - loss: 2.3026 - accuracy: 0.2656
Train on 64 samples
64/64 [==============================] - 0s 8ms/sample - loss: 2.2676 - accuracy: 0.2656
Train on 64 samples
64/64 [==============================] - 0s 8ms/sample - loss: 2.2667 - accuracy: 0.2969
Train on 64 samples
64/64 [==============================] - 0s 8ms/sample - loss: 2.2315 - accuracy: 0.2031
Train on 64 samples
64/64 [==============================] - 0s 8ms/sample - loss: 2.3026 - accuracy: 0.2188
Train on 64 samples
64/64 [==============================] - 0s 8ms/sample - loss: 2.2668 - accuracy: 0.2344
Train on 64 samples
64/64 [============

In [9]:
net.load_weights('googlenet_weights.h5')
X_test, y_test = dataLoader.get_batch_test(2000)
net.evaluate(X_test, y_test, verbose=2)

2000/2000 - 7s - loss: 2.2829 - accuracy: 0.2270


[2.2828662757873537, 0.227]